# Summary

A. [Getting datasets](#A.-Getting-data)
- text files stored locally
- text files from a website

B. [Cleaning and transforming datasets](#B.-Cleaning-and-transforming-data)
- Inspecting the data frame
- Selecting a subset of a data frame
- Augmenting a data frame with additional columns/rows (or replacing existing columns/rows)
- Dealing with missing values
- Converting between the wide and long forms of a data frame
- Sorting a data frame

C. [Exploring and visualizing datasets](#C.-Exploring-and-visualizing)
- Summary statistics and frequencies
- Scatter plots & regression lines
- Histograms & density lines
- Boxplots & barplots with error bars

# A. Getting data

Data can come from different sources, e.g.:
- text files stored locally (with different extensions, e.g., `.txt` or `.csv`)
- text files from a website

#### Different functions to read data from file:
- `read.table` is the principal and more general means of reading tabular data into R
- `read.csv` sets the default separator to a comma, and assumes that the data has a header row
- `read.csv2` is its European cousin, using a comma for decimal places and a semicolon as a separator
- `read.delim` and `read.delim2` import tab-delimited files with full stops or commas for decimal places, respectively

#### Main arguments that you might want to tweak:
- `header` are the names of the variables its first line?
- `sep` how are the values separated?
- `dec` how are the decimals coded? (dots or commas)
- `na.strings` how are the missing values coded?

In [ ]:
# Example of tab delimitated data
data <- read.table('~/git/RcourseSpring2019/data/data_taskA.txt')
head(data)

In [ ]:
# Example of comma delimitated data, with header as first line
data <- read.table('~/git/RcourseSpring2019/data/data_wpa4.csv', 
                   sep=',',
                   header=TRUE)
head(data)

In [ ]:
# Example of ; delimitated data, with header as first line
data <- read.table('~/git/RcourseSpring2019/data/student-por.csv',
                   sep=';',
                   header=TRUE)
head(data)

In [ ]:
# Example of data from github (remember to get the raw data first):
data <- read.csv('https://raw.githubusercontent.com/laurafontanesi/RcourseSpring2019/master/data/matthews_demographics.csv',
                 )
head(data)

# Try out on your own with data from: https://github.com/BuzzFeedNews/zika-data/tree/master/data/parsed

In [ ]:
# How to first download a file and then load it in R:
data_url <- "http://samplecsvs.s3.amazonaws.com/Sacramentorealestatetransactions.csv"
local_file_location <- "~/git/RcourseSpring2019/data/Sacramentorealestatetransactions.csv"

download.file(data_url, local_file_location)

data <- read.csv(local_file_location)
head(data)

# B. Cleaning and transforming data

Much of the task of cleaning data involves manipulating data frames to get them into the desired form:

0. Inspecting the data frame
1. Selecting a subset of a data frame
2. Augmenting a data frame with additional columns/rows (or replacing existing columns/rows)
3. Dealing with missing values
4. Converting between the wide and long forms of a data frame
5. Sorting a data frame

## 1. Inspecting the data frame

In [ ]:
# Create a fake data.frame
N = 25
fake_data <- data.frame(
    participant = 1:N,
    risk_parameter = round(rgamma(N, 1, 1), 1)
)

fake_data[fake_data$risk_parameter > 1, "risk_preference"] = "risk_seeking"
fake_data[fake_data$risk_parameter < 1, "risk_preference"] = "risk_avoidant"
fake_data[fake_data$risk_parameter == 1, "risk_preference"] = "risk_neutral"

fake_data$age = round(22 + rnorm(N, -fake_data$risk_parameter, .8))

fake_data

In [ ]:
# Get the row and columns names, and the number of rows and columns:
dimnames(fake_data)
dim(fake_data)

In [ ]:
# Careful that:
length(fake_data)

In [ ]:
# Inspect head and tail:
head(fake_data)
tail(fake_data)

In [ ]:
summary(fake_data)

## 2. Subsetting:

In [ ]:
# Index a specific cell:
fake_data[6, "risk_parameter"]

# or a row:
fake_data[6,]

# or a column:
fake_data[,"risk_parameter"]

In [ ]:
# Get rows based on a logical condition (aka Boolean indexing):
fake_data[fake_data$risk_preference == 'risk_avoidant',]

fake_data[fake_data$risk_parameter < 1,]

# and maybe select only few columns to keep:
subset(fake_data, 
       age >= 22, 
       select=c('participant', 'age'))

In [ ]:
# Get everything apart from the first column:
head(fake_data[,-1])

# or third column:
head(fake_data[,-3])

# get specific columns based on the names:
columns_to_keep = c('participant', 'age')
head(fake_data[,columns_to_keep])

## 3. Adding columns and rows:
The usual form to add a new column is:

data.frame.name$column.name <- vector,

where the vector is either:
- length = number of rows
- lenght = 1 (in that case the same value is assigned to every row)
- lenght < number of rows, where number of rows is a multiple of length (in that case the vector is repeated until the whole column is filled)

Otherwise, you can use `cbind` and `merge`.

In [ ]:
# Load some example data:
data <- read.table('~/git/RcourseSpring2019/data/data_taskA.txt')
row.names(data) <- NULL
head(data)
tail(data)

In [ ]:
# Add a column:
data$RT_sec <- data$RT_msec/1000
data$conditionA <- 'difficult'
data$conditionB <- c('difficult', 'easy')

head(data)

tail(data)

In [ ]:
# But:
data$conditionC <- c('a', 'b', 'c', 'd', 'e')

To add rows, you simply need to index a "new line" (as with adding columns), by either the row number or row name.

Otherwise, you can simply use `rbind`.

In [ ]:
# Add a row with values for specific columns:
new_row <- dim(data)[1] + 1
new_row

data[new_row, c(1, 2)] <- c(50, 'female')

tail(data)

In [ ]:
# Add a row with the same value for all columns:
new_row <- new_row + 1
new_row

data[new_row,] <- NA

tail(data)

## 4. Missing values

In [ ]:
# Remove any row containing missing values:
data_noNA <- na.omit(data)

tail(data_noNA)

nrows_with_NA <- dim(data)[1] - dim(data_noNA)[1]
nrows_with_NA

In [ ]:
# Remove any row contanining missed participant number:
data_noNA_participant <- data[!is.na(data$participant),]

tail(data_noNA_participant)

nrows_with_NA_participant <- dim(data)[1] - dim(data_noNA_participant)[1]
nrows_with_NA_participant

In [ ]:
# Remove any column containing missing values (in this case it doesn't make much sense, but might be useful to know):
which_NAs <- apply(
    X = is.na(data_noNA_participant),
    MARGIN = 2,
    FUN = sum
)

data_noNA_participant <- data_noNA_participant[,which_NAs == 0]

head(data_noNA_participant)

## 5. Converting between the wide and long forms of a data frame

The data shown before are in the **long** format: repeated measurements from each participants are stored in different rows. This format is relatively common in cognitive psychology, as we often present participants with tasks in which they have to provide multiple responses for the same item/condition.

Another common format is the **wide** format: here, participants' repeated responses will be in a single row, and each response is in a separate column.

In [ ]:
# Create fake data:
N = 10
mean_score_students = runif(N, 5, 10)
fake_data_wide <- data.frame(
    student = 1:N,
    age = round(rnorm(N, 30, 5)),
    score_WPA1 = mean_score_students,
    score_WPA2 = mean_score_students*0.9 + rnorm(N, 0, 0.1),
    score_WPA3 = mean_score_students*0.5 + rnorm(N, 0, 1),
    score_WPA4 = mean_score_students*0.7 + rnorm(N, 0, 0.2)
)

fake_data_wide

In [ ]:
# Load library reshape2 to easily transform to long format:
library(reshape2)
fake_data_long <- melt(fake_data_wide, id.vars = "student", measure.vars = c("score_WPA1", "score_WPA2", "score_WPA3", "score_WPA4"))

# Re-order based on student number:
fake_data_long[order(fake_data_long$student),]

In [ ]:
# To go back to wide format, use dcast and acast (see the difference):
dcast(
    fake_data_long,
    student ~ variable
     )

acast(
    fake_data_long,
    student ~ variable
     )

In [ ]:
# What if you just want to have the unique value per participant of 1 variable, e.g., age? 

# When data is in the wide format:
age_df <- aggregate(age ~ student, data = fake_data_wide, FUN=unique)
age_df

In [ ]:
# But it's actually the same for long format:
age_df <- aggregate(age ~ participant, data = data, FUN=unique)
age_df

In [ ]:
# Alternatively, we can use the dplyr package: https://dplyr.tidyverse.org/
library(dplyr)

grouped_data = group_by(data, participant)

mean_performance = summarise(grouped_data, mean_accuracy=mean(accuracy, na.rm=TRUE), mean_rt=mean(RT_sec, na.rm=TRUE), age=unique(age))

In [ ]:
mean_performance

## 6. Sorting the data frame by value:

In [ ]:
# The most usual way to do this is by using the function order:
age_df[order(age_df$age),]

In [ ]:
# Or you can use the custom function arrange from the plyr library (easier syntax...):

library(plyr)

arrange(age_df, age)

# C. Exploring and visualizing

1. Summary statistics and frequencies
2. Scatter plots & regression lines
3. Histograms & density lines
4. Boxplots & barplots with error bars

## 1. Summary statistics and frequencies:

In [ ]:
data <- read.csv(local_file_location)
head(data)
summary(data)
dim(data)

In [ ]:
# The single functions for the summary statistics above would be:
range(data$price)
min(data$price)
max(data$price)

quantile(data$price, c(.25, .5, .75))

median(data$price)

mean(data$price)

In [ ]:
# IQR wraps quantile to give the interquartile range (the 75th percentile minus the 25th percentile):
IQR(data$price)

In [ ]:
# You can use with to make the syntax a bit more digestible:
with(data, mean(price))

In [ ]:
# Careful that:
colMeans(data)

In [ ]:
# So you should only select numberical variables:
colMeans(data[,c('beds', 'baths', 'sq__ft', 'price')])

In [ ]:
# A more general function is apply, that lets you apply any function to either columns or rows of a dataframe:
apply(
    data[,c('beds', 'baths', 'sq__ft', 'price')],
    MARGIN = 2, # 2 is for columns, 1 is for rows
    FUN = mean
)

apply(
    data[,c('beds', 'baths', 'sq__ft', 'price')],
    MARGIN = 2, # 2 is for columns
    FUN = sd
)

In [ ]:
# Similarly, for frequencies:
table(data)

In [ ]:
# Here you want to select categorical variables:
table(data[,c('city', 'type')])

In [ ]:
# You can also combine cut with bins, to get the frequencies per bin of a certain variable.

# For example, we can look how many properties are there depending on the squared meters, in bins of 1000:
table(cut(data$sq__ft, seq.int(0, 6000, 1000)))

In [ ]:
# You can also calculate correlations and covariances:
with(data, cor(price, sq__ft))

with(data, cov(price, sq__ft))

In [ ]:
# But if you want a correlation or covariance matrix you should use:
cor_mat <- round(cor(data[,c('price', 'sq__ft', 'beds', 'baths')]), 2)
cor_mat

cov_mat <- cov(data[,c('price', 'sq__ft', 'beds', 'baths')])
cov_mat

## 2. Scatter plots & regression lines:

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

plot(x = data$sq__ft, 
     y = data$price, 
     col = rgb(.1, .1, .1, .3), 
     pch = 20,
     xlab = 'Squared feet',
     ylab = 'Price',
     bty = 'l'
    )

abline( # add regression line
    lm(price ~ sq__ft, data),
    col = 'maroon4',
    lw = 2
)

In [ ]:
# From this plot we see that there are a lot of 0 squared feet houses... 
# this does not seem to be plausible.
# Let's fit a regression without these houses.

options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

plot(x = data$sq__ft, 
     y = data$price, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     xlab = 'Squared feet',
     ylab = 'Price',
     bty = 'l'
    )



# Model 1:
lm_1 <- lm(price ~ sq__ft, data) # calculate the regression model
slope_1 <- lm_1$coefficients['sq__ft'] # extract the slope for the label positioning 
intercept_1 <- lm_1$coefficients['(Intercept)'] # extract the intercept for the label positioning 

abline( # add regression line
    lm_1,
    col = 'maroon4',
    lw = 2
)
text(x = max(data$sq__ft)*.9, # add label at the end of the regression line
     y = slope_1 * max(data$sq__ft) + intercept_1, 
     "Including 0", 
     srt=0.2)



# Model 2:
lm_2 <- lm(price ~ sq__ft, data[data$sq__ft > 0,]) # calculate the regression model
slope_2 <- lm_2$coefficients['sq__ft'] # extract the slope for the label positioning 
intercept_2 <- lm_2$coefficients['(Intercept)'] # extract the intercept for the label positioning

abline( # add regression line
    lm_2,
    col = 'maroon1',
    lw = 2
)
text(x = max(data$sq__ft)*.9,  # add label at the end of the regression line
     y = slope_2 * max(data$sq__ft) + intercept_2, 
     "Excluding 0", 
     srt=0.2)

In [ ]:
# Good checks to do: residual plot
options(repr.plot.width=10, repr.plot.height=5) # this is not necessary in RStudio.

# First calculate the residuals for both models
residuals_1 <- resid(lm_1)
residuals_2 <- resid(lm_2)

# We should change the layout: we want now the two models' residuals next to each other.
# We can use the mfrow argument in the par function, that is a general plotting function for changing standard
# plotting parameters. This means that we should always remember to change it back to the default at the 
# end of the plotting.
par(mfrow = c(1, 2))

plot(x = data[, 'sq__ft'], 
     y = residuals_1, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     main = 'Including 0',
     xlab = 'Squared feet',
     ylab = 'Residuals',
     bty = 'l'
    )
abline(h=0, lty=3) # plot a dotted horizontal line at 0

plot(x = data[data$sq__ft > 0, 'sq__ft'], 
     y = residuals_2, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     main = 'Excluding 0',
     xlab = 'Squared feet',
     ylab = 'Residuals',
     bty = 'l'
    )
abline(h=0, lty=3) # plot a dotted horizontal line at 0

# Reset layout:
par(mfrow = c(1, 1))

In [ ]:
# The residuals do not seem to be exactly normally distributed.
# Let's try to log transform the data (both x and y).

options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

data_excluding0 = data[data$sq__ft > 0,]
data_excluding0$log_price = log(data_excluding0$price)
data_excluding0$log_sq__ft = log(data_excluding0$sq__ft)

plot(x = data_excluding0$log_sq__ft, 
     y = data_excluding0$log_price, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     xlab = 'Log squared feet',
     ylab = ' Log price',
     bty = 'l',
     log = 'xy'
    )

In [ ]:
# Although the data seem now much more suitable for linear regression, there seem to be an outlier:
data_excluding0[data_excluding0$log_price < 10,]

# Because a a price of 2000 also seems quite unrealistic, we can just exclude this property:
data_excluding0 <- data_excluding0[data_excluding0$log_price > 10,]

In [ ]:
# Let's try again, this time refitting the regression and checking the residuals:

options(repr.plot.width=10, repr.plot.height=5) # this is not necessary in RStudio.

par(mfrow = c(1, 2))

plot(x = data_excluding0$log_sq__ft, 
     y = data_excluding0$log_price, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     xlab = 'Log squared feet',
     ylab = ' Log price',
     bty = 'l',
    )

lm_3 <- lm(log_price ~ log_sq__ft, data_excluding0) # calculate the regression model
residuals_3 <- resid(lm_3) # calculate residuals

abline( # add regression line
    lm_3,
    col = 'maroon',
    lw = 2
)

plot(x = data_excluding0$log_sq__ft, 
     y = residuals_3, 
     col = rgb(0.1, 0.1, 0.1, .3), 
     pch = 20,
     xlab = 'Log squared feet',
     ylab = 'Residuals',
     bty = 'l'
    )
abline(h=0, lty=3) # plot a dotted horizontal line at 0

par(mfrow = c(1, 1))

# Much better!

In [ ]:
# Lastly, we might want to now represent a third variable as a color code, e.g., the number of bathrooms:

# And now let's plot a scatter plot where the color indicates the number of bathrooms:
options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

plot(x = data_excluding0$log_sq__ft, 
     y = data_excluding0$log_price, 
     col = data_excluding0$baths, 
     pch = 20,
     xlab = 'Log squared feet',
     ylab = ' Log price',
     bty = 'l',
    )

In [ ]:
# The colors are not the best :D
# Here some code to make it slightly more pleasing to the eyes:

options(repr.plot.width=4, repr.plot.height=3) # this is not necessary in RStudio.

# Let's first build a palette with number of colors the levels in he baths variable:
colors <- c()
gradient <- 0
for (c in unique(data_excluding0$baths)) {
    colors <- c(colors, rgb(gradient, .1, gradient, .3))
    gradient <- gradient + .25
}

# And show the palette (just for fun):
image(1:length(colors), 
      1, 
      as.matrix(1:length(colors)), 
      col=colors,
      main = 'Number of bathrooms',
      xlab="", ylab = "", xaxt = "n", yaxt = "n", bty = "n")

for (c in unique(data_excluding0$baths)) {
    text(c, 1, c)
}

# Now we assign a color to each data point based on the palette we created:
zcolor <- colors[data_excluding0$baths]

# And now let's plot a scatter plot where the color indicates the number of bathrooms:
options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

plot(x = data_excluding0$log_sq__ft, 
     y = data_excluding0$log_price, 
     col = zcolor, 
     pch = 20,
     xlab = 'Log squared feet',
     ylab = ' Log price',
     bty = 'l',
    )

## 3. Histograms & density lines:

What we saw in this scatteplot and regression exercise, is that it is important to inspect the **distribution** of the data, before we even try to attempt to fit models (as most statistical models assume the data to be distributed in certain ways...).

In [ ]:
options(repr.plot.width=10, repr.plot.height=5) # this is not necessary in RStudio.

hist(data$sq__ft,
     main = '',
     xlab = 'Squared feet',
     breaks = 50
    )

From this plot would have been already easy to **conclude** that:
- there are a lot of data to be excluded, having 0 squared feet
- the remaining data are positively skewed (therefore might be a good idea to log transform)

You can also help the interpretation by plotting **mean** and **median** as vertical lines.

When the median is lower than the mean, we are likely to have positively skewed data:

In [ ]:
options(repr.plot.width=8, repr.plot.height=5) # this is not necessary in RStudio.

h <- hist( # assign it to variable to be able to access the frequencies
    data$sq__ft,
    main = '',
    xlab = 'Squared feet',
    breaks = 50
)

abline( # add the mean (calculated excluding the 0s)
    v=mean(data[data$sq__ft > 0, 'sq__ft']),
    col='blue3',
    lty=3,
    lw=3
      )
abline( # add the median (calculated excluding the 0s)
    v=median(data[data$sq__ft > 0, 'sq__ft']),
    col='blue3',
    lty=2,
    lw=3
      )

# To better position the legend, we can get the max frequency in the histogram:
max_frequency <- max(h$counts)

legend(
    max(data$sq__ft)*.6, 
    max_frequency, 
    legend=c("Mean", "Median"),
    col='blue3',
    lty=c(3, 2), 
    bg='white', 
    lw=3)

To better control the binning of the histograms, we should give a specific vector to the **breaks** argument, instead of just a number:

In [ ]:
options(repr.plot.width=4, repr.plot.height=4) # this is not necessary in RStudio.

# So, instead of:

hist(
    data$baths,
    main = '',
    xlab = 'Number of Bathrooms',
    breaks = 6
)

# We do:

hist(
    data$baths,
    main = '',
    xlab = 'Number of Bathrooms',
    breaks = seq(-.5, max(data$baths)+.5, 1)
)

# Not only in the top histogram there are 5 instead of the specified 6 bins, but it's unclear if the first bar
# includes 0 or also 1s...
# So always specify the breaks as a vector to increase interpretability!

Instead of the frequencies, we often want to plot **densities**.

In [ ]:
options(repr.plot.width=10, repr.plot.height=5) # this is not necessary in RStudio.

better_color <- rgb(.2, .4, .5, .3) # Già che ci siamo, we also change the color

hist(
    data_excluding0$price,
    main = '',
    xlab = 'Price',
    breaks = 50,
    freq = FALSE, # set this to false to get densities,
    col = better_color,
    border = better_color # you can change the color of the border to make it nicer...
)

# So that's good, but it's also super nice to add a density line on top of the histogram:
density_price <- density(data_excluding0$price)

better_color_denser <- rgb(.2, .4, .5, .8)

lines(
    density_price,
    lw=3,
    col= better_color_denser
    )

In [ ]:
# Sometimes we want to compare distributions with each other:
col_1 <- rgb(.5, .3, .2, .3)
col_2 <- rgb(.2, .5, .2, .3)

breaks <- seq(min(data_excluding0$price), max(data_excluding0$price)*1.1, 50000)

options(repr.plot.width=10, repr.plot.height=6) # this is not necessary in RStudio.

hist(
    data_excluding0$price[data_excluding0$beds <= 3],
    main = '',
    xlab = 'Price',
    breaks = breaks,
    freq = FALSE,
    col = col_1,
    border = col_1
)

hist(
    data_excluding0$price[data_excluding0$beds > 3],
    main = '',
    xlab = 'Price',
    breaks = breaks,
    freq = FALSE,
    col = col_2,
    border = col_2,
    add = TRUE # set this to true to overlap!
)

legend('topright', 
       legend = c("Up to 3", "More than 3"), 
       col = c(col_1, col_2), 
       pch = c(15, 15))

lines(
    density(data_excluding0$price[data_excluding0$beds <= 3]),
    lw=3,
    col= col_1
    )

lines(
    density(data_excluding0$price[data_excluding0$beds > 3]),
    lw=3,
    col= col_2
    )

## 4. Boxplots & barplots with error bars:

The plots above are very good when we have continuous variables. Often though, we have continuous measurements for different experimental conditions, or groups of subjects, etc. With 2 or 3 levels of such variables, it's still relatively convenient to plot overlapping histograms or density lines alone. But when the number of levels increases, we might want to switch to **boxplots**:

In [ ]:
options(repr.plot.width=6, repr.plot.height=5) # this is not necessary in RStudio.

# Example:
boxplot(
    price ~ baths, 
    data = data_excluding0,
    xlab = 'Number of bathrooms',
    ylab = 'Price'
)

In the plot above, it is not necessarily clear how many observations we have per level of the categorical variable. To explore that, we can use **barplots**.

Remember that **barplots** are a bit more stupid than **histograms** and **boxplots**. They often require us to calculate stuff first:

In [ ]:
# How many properties are there per level of bedrooms?
table_beds <- table(data_excluding0$beds) # Calculate the y values

barplot(
    table_beds,
    col = better_color,
    border = better_color,
    ylab = 'Counts',
    xlab = 'Number of bedrooms'
)

In [ ]:
# When we have a lot of categorical variables, it's nice to make it horizontal, 
# and order it by frequency:

options(repr.plot.width=10, repr.plot.height=9) # this is not necessary in RStudio.

table_city <- table(data_excluding0$city) # Calculate the y values
table_city <- sort(table_city) # Order it by frequency

par(las=1, # make the labels horizontal
    mar=c(5.1,7.1,4.1,2.1)) # increase left margin

barplot(
    table_city,
    col = better_color,
    border = better_color,
    xlab = 'Counts',
    ylab = '',
    horiz = TRUE,
    cex.names = .6
)

**barplots** are also very flexible. Another important thing we might want to do is to for example compare the mean of X aggregated by either 1 or two independent variables.

In [ ]:
# Data retrieved from https://www.kaggle.com/ronitf/heart-disease-uci
data <- read.csv('~/git/RcourseSpring2019/data/heart.csv')
head(data)

#### One grouping variable, confidence intervals:

In [ ]:
# Plot the mean cholestoral per sex(1 = male; 0 = female), with CONFIDENCE INTERVALS:

# 1. First calculate the grouped mean, steandard deviation, and number of observations
grouped_mean <- aggregate(
    formula = chol ~ sex,
    data = data,
    FUN = mean
)

grouped_sd <- aggregate(
    formula = chol ~ sex,
    data = data,
    FUN = sd
)

grouped_n <- aggregate(
    formula = chol ~ sex,
    data = data,
    FUN = length
)

# 2. Then calculate the standard error of the grouped means:

# It is the standard deviation of the vector sampling distribution. 
# Calculated as the SD divided by the square root of the sample size. 
# By construction, SE is smaller than SD. With a very big sample size, SE tends toward 0.

grouped_se <- grouped_sd$chol / sqrt(grouped_n$chol)
grouped_se

# 3. Then calculate the confidence intervals:

# This interval is defined so that there is a specified probability that a value lies within it. 
# It is calculated as t * SE. Where t is the value of the Student’s t-distribution for a specific alpha. 
# Its value is often rounded to 1.96 (its value with a big sample size). 
# If the sample size is huge or the distribution not normal, it is better to calculate the CI using the bootstrap method.

alpha <- 0.05 # Significance level
t <- qt((1-alpha)/2 + .5, grouped_n$chol - 1)   # tend to 1.96 if sample size is big enough
grouped_ci <- t*grouped_se
grouped_ci

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) # this is not necessary in RStudio.

b <- barplot(
    height = grouped_mean$chol,
    names.arg = grouped_mean$sex,
    col = better_color,
    border = better_color,
    ylim = c(0, max(grouped_mean$chol+grouped_ci)),
    xlab = 'Sex',
    ylab = 'Serum cholestoral in mg/dl'
       )

segments(
    x0 = b, 
    x1 = b, 
    y0 = grouped_mean$chol-grouped_ci, 
    y1 = grouped_mean$chol+grouped_ci, 
    lwd = 2
)

#### One grouping variable, Standard errors:

In [ ]:
# Plot the mean resting maximum heart rate per age, with STANDARD ERRORS:

# 1. First calculate the grouped mean, steandard deviation, and number of observations
grouped_mean <- aggregate(
    formula = thalach ~ age,
    data = data,
    FUN = mean
)

grouped_sd <- aggregate(
    formula = thalach ~ age,
    data = data,
    FUN = sd
)

grouped_n <- aggregate(
    formula = thalach ~ age,
    data = data,
    FUN = length
)

# 2. Then calculate the standard error of the grouped means:

grouped_se <- grouped_sd$thalach / sqrt(grouped_n$thalach)

# Eliminate the NA:
grouped_mean <- grouped_mean[!is.na(grouped_se),]
grouped_se <- grouped_se[!is.na(grouped_se)]

In [ ]:
options(repr.plot.width=9, repr.plot.height=5) # this is not necessary in RStudio.

b <- barplot(
    height = grouped_mean$thalach,
    names.arg = grouped_mean$age,
    col = better_color,
    border = better_color,
    ylim = c(0, max(grouped_mean$thalach + grouped_se)),
    xlab = 'Age',
    ylab = 'Maximum heart rate '
       )

segments(
    x0 = b, 
    x1 = b, 
    y0 = grouped_mean$thalach - grouped_se, 
    y1 = grouped_mean$thalach + grouped_se, 
    lwd = 2
)

#### Two grouping variables, confidence intervals:

In [ ]:
# 1. First calculate the grouped mean, steandard deviation, and number of observations
grouped_mean <- aggregate(
    formula = trestbps ~ sex + cp,
    data = data,
    FUN = mean
)

grouped_sd <- aggregate(
    formula = trestbps ~ sex + cp,
    data = data,
    FUN = sd
)

grouped_n <- aggregate(
    formula = trestbps ~ sex + cp,
    data = data,
    FUN = length
)

head(grouped_mean)

In [ ]:
# To feed into the barplot function, we need a table-like object, with, e.g, age as rows and sex as columns.
# We can thus use the reshape2 package for this purpose and transform our data to wide format using the function acast:

grouped_mean_wide <- acast(
    data = grouped_mean, 
    formula = sex ~ cp
)

grouped_sd_wide <- acast(
    data = grouped_sd, 
    formula = sex ~ cp
)

grouped_n_wide <- acast(
    data = grouped_n, 
    formula = sex ~ cp
)

grouped_mean_wide

In [ ]:
# 2. Then calculate the standard error of the grouped means:
grouped_se_wide <- grouped_sd_wide / sqrt(grouped_n_wide)

# 3. Then calculate the confidence intervals:

alpha <- 0.05 # Significance level
t <- qt((1-alpha)/2 + .5, grouped_n_wide - 1)   # tend to 1.96 if sample size is big enough
grouped_ci_wide <- t*grouped_se_wide

grouped_ci_wide

In [ ]:
# Eliminate NAs:
which_NAs <- apply(
    X = is.na(grouped_ci_wide),
    MARGIN = 2,
    FUN = sum
)

grouped_mean_wide <- grouped_mean_wide[,which_NAs == 0]
grouped_ci_wide <- grouped_ci_wide[,which_NAs == 0]

In [ ]:
# Now we are ready to plot:
b <- barplot(
    height = grouped_mean_wide,
    names.arg = c('typical angina', 'atypical angina', 'non-anginal pain', 'asymptomatic'),
    ylab = 'Resting blood pressure',
    xlab = 'Chest pain type',
    col = c('pink3', 'lightblue3'),
    beside = TRUE, 
    legend.text = TRUE,
    ylim = c(0, max(grouped_mean_wide + grouped_ci_wide))
)

segments(
    x0 = b,
    x1 = b,
    y0 = grouped_mean_wide - grouped_ci_wide,
    y1 = grouped_mean_wide + grouped_ci_wide,

)

#### Two grouping variables, standard errors:

In [ ]:
# 1. First calculate the grouped mean, steandard deviation, and number of observations
grouped_mean <- aggregate(
    formula = chol ~ sex + age,
    data = data,
    FUN = mean
)

grouped_sd <- aggregate(
    formula = chol ~ sex + age,
    data = data,
    FUN = sd
)

grouped_n <- aggregate(
    formula = chol ~ sex + age,
    data = data,
    FUN = length
)

head(grouped_mean)

In [ ]:
grouped_mean_wide <- acast(
    data = grouped_mean, 
    formula = sex ~ age
)

grouped_sd_wide <- acast(
    data = grouped_sd, 
    formula = sex ~ age
)

grouped_n_wide <- acast(
    data = grouped_n, 
    formula = sex ~ age
)

grouped_mean_wide

In [ ]:
# 2. Then calculate the standard error of the grouped means:
grouped_se_wide <- grouped_sd_wide / sqrt(grouped_n_wide)

In [ ]:
# Eliminate NAs:
which_NAs <- apply(
    X = is.na(grouped_se_wide),
    MARGIN = 2,
    FUN = sum
)

grouped_mean_wide <- grouped_mean_wide[,which_NAs == 0]
grouped_se_wide <- grouped_se_wide[,which_NAs == 0]

In [ ]:
# Now we are ready to plot:
b <- barplot(
    height = grouped_mean_wide,
    ylab = 'Serum cholestoral in mg/dl',
    xlab = 'Age',
    col = c('pink3', 'lightblue3'),
    beside = TRUE, 
    legend.text = TRUE,
    ylim = c(0, max(grouped_mean_wide + grouped_se_wide))
)

segments(
    x0 = b,
    x1 = b,
    y0 = grouped_mean_wide - grouped_se_wide,
    y1 = grouped_mean_wide + grouped_se_wide,

)